# Movie Review Analysis Outline
## import necessary python libraries


In [1]:
# output.txt is created by running a python script to convert the format of the movie data
# into a format that spark readily converts into a RDD

# "transformData.py" is this script which is intended to be run once on our data
import re
import string


## Import the Data
## Clean up Data (extract data we need (i.e. stars, and perhaps text of review)

In [15]:
def strip_html_tags(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data.lower())

# CHANGED SEPARATOR TO THREE /// because / separator was causing discrepencies
# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky
movies_txt = sc.textFile("output.txt").map(lambda x: (cleanData(x).split('///')))

#TODO
#parse and take care of funky symbols in summary and text <- normalize to root words maybe?
def cleanData (data):
    data = strip_html_tags(data)
    data =  re.sub("[\.\t\,\:;\(\)\.\"\'\~\-\!\?\`]", "",data, 0, 0)
    return data

#re.sub("[\.\t\,\:;\(\)\.]", " ", strip_html_tags(data.lower()), 0, 0)

#testing clean data
print cleanData("""
"there is so much darkness now ~ come for the miracle"', 'synopsis: on the daily trek from juarez, mexico to el paso, texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. investigative reporter karina danes (minnie driver) arrives from los angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br ', '><br ', '>her story goes nationwide when a young girl named mariela (ana claudia talancon) survives a vicious attack and walks out of the desert crediting the blessed virgin for her rescue. her story is further enhanced when the "wounds of christ" (stigmata) appear in her palms. she also claims to have received a message of hope for the virgin mary and soon a fanatical movement forms around her to fight against the evil that holds such a stranglehold on the area.<br ', '><br ', ">critique: possessing a lifelong fascination with such esoteric matters as catholic mysticism, miracles and the mysterious appearance of the stigmata, i was immediately attracted to the '05 dvd release""")


there is so much darkness now  come for the miracle synopsis on the daily trek from juarez mexico to el paso texas an ever increasing number of female workers are found raped and murdered in the surrounding desert investigative reporter karina danes minnie driver arrives from los angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truthher story goes nationwide when a young girl named mariela ana claudia talancon survives a vicious attack and walks out of the desert crediting the blessed virgin for her rescue her story is further enhanced when the wounds of christ stigmata appear in her palms she also claims to have received a message of hope for the virgin mary and soon a fanatical movement forms around her to fight against the evil that holds such a stranglehold on the areacritique possessing a lifelong fascination with such esoteric matters as catholic mystic

In [16]:
# clean the data for each element of each sub list
def prepData (list_str):
    L= []
    for x in list_str:
        #print x
        L.append(cleanData(x))
    return L

# [u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text']
# headers for the project
movies_new = movies_txt.map(lambda L: (L[0], L[4], L[6], L[7]) if len(L) == 8 else ("ERROR"))
removeHTMLTags = movies_new.map(prepData); 
print (removeHTMLTags.collect())

[[u'productid', u'score', u'summary', u'text'], [u'b003ai2vga', u'30', u'there is so much darkness now  come for the miracle', u'synopsis on the daily trek from juarez mexico to el paso texas an ever increasing number of female workers are found raped and murdered in the surrounding desert investigative reporter karina danes minnie driver arrives from los angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truthher story goes nationwide when a young girl named mariela ana claudia talancon survives a vicious attack and walks out of the desert crediting the blessed virgin for her rescue her story is further enhanced when the wounds of christ stigmata appear in her palms she also claims to have received a message of hope for the virgin mary and soon a fanatical movement forms around her to fight against the evil that holds such a stranglehold on the areacritique pos

## Divide into 2 datasets (one for training and one for testing) 

## Train classifier

In [ ]:
# train the classifier 



## Run classifier on testing set